In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

SETTING = kraft.read_json("../project.json")

if "project_directory_path" in SETTING:

    SETTING.update(
        kraft.read_json(os.path.join(SETTING["project_directory_path"], "project.json"))
    )

    SETTING["output_directory_path"] = os.path.join(
        SETTING["project_directory_path"], "output"
    )

PATH = make_path_dict(SETTING)

In [ ]:
w = pd.read_csv(PATH["w.tsv"], sep="\t", index_col=0)

h = pd.read_csv(PATH["h.tsv"], sep="\t", index_col=0)

w.columns.name = h.index.name

h.columns.name = SETTING["sample_alias"]

In [ ]:
for w_or_h, dataframe, normalization_axis in (("w", w, 1), ("h", h, 0)):

    dataframe = kraft.normalize_dataframe(dataframe, normalization_axis, "-0-")

    dataframe = dataframe.iloc[
        kraft.cluster_matrix(dataframe.values, 0),
        kraft.cluster_matrix(dataframe.values, 1),
    ]

    kraft.plot_heat_map(
        dataframe,
        layout={
            "height": 800,
            "width": 800,
            "title": {"x": 0.5, "text": w_or_h.title()},
            "xaxis": {"showticklabels": w_or_h == "w"},
            "yaxis": {"showticklabels": w_or_h == "h"},
        },
        html_file_path=os.path.join(
            PATH["{}/".format(w_or_h)], "{}.html".format(w_or_h)
        ),
    )

In [ ]:
for element_type, w_or_h, size_normalization_axis in (
    ("feature", "w", 1),
    ("sample", "h", 0),
):

    elements_to_peek = SETTING["{}s_to_peek".format(element_type)]

    if element_type == "feature":

        dataframe = w.loc[
            [
                feature
                for feature in w.index
                if any(
                    feature_to_peek in feature for feature_to_peek in elements_to_peek
                )
            ]
        ]

    elif element_type == "sample":

        dataframe = h[
            [
                sample
                for sample in h.columns
                if any(sample_to_peek in sample for sample_to_peek in elements_to_peek)
            ]
        ]

    if dataframe.empty:

        continue

    kraft.plot_bubble_map(
        dataframe,
        dataframe_color=dataframe,
        html_file_path=os.path.join(
            PATH["{}/".format(w_or_h)], "{}.peek.html".format(w_or_h)
        ),
    )

In [ ]:
signal = pd.read_csv(PATH["feature_x_sample.signal.tsv"], sep="\t", index_col=0)

signal.columns.name = SETTING["sample_alias"]

signal.fillna(value=0, inplace=True)

In [ ]:
feature_x_sample_processed = pd.read_csv(
    PATH["feature_x_sample.processed.tsv"], sep="\t", index_col=0
)

feature_x_sample_processed.columns.name = SETTING["sample_alias"]

In [ ]:
for targets, data_signal, data_raw in (
    (w.T, signal.T, feature_x_sample_processed.T),
    (h, signal, feature_x_sample_processed),
):

    kraft.make_match_panels(
        targets,
        {
            "Signal": {"dataframe": data_signal, "data_type": "continuous"},
            "Feature-x-Sample": {"dataframe": data_raw, "data_type": "continuous"},
        },
        n_required_for_match_function=SETTING["n_required_for_match_function"],
        n_sampling=0,
        n_permutation=0,
        plot_std=SETTING["plot_std"],
    )